<a href="https://colab.research.google.com/github/ahamedfoisal/data-feature/blob/main/Weather_and__workout_api_data_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import re
import requests
from google.colab import userdata


# Wger authentication (for private endpoints, public endpoints don't need this)
WGER_USERNAME = 'ahamedfoisal'
WGER_PASSWORD = userdata.get('wger_password')
WEATHER_API_KEY = userdata.get('weather_api')

# Function to remove HTML tags from the exercise description
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def get_wger_token():
    # Get the JWT access token for Wger API
    token_url = 'https://wger.de/api/v2/token'
    response = requests.post(token_url, data={'username': WGER_USERNAME, 'password': WGER_PASSWORD})
    if response.status_code == 200:
        tokens = response.json()
        access_token = tokens['access']
        return access_token
    else:
        print("Error getting Wger token")
        return None

# Step 1: Get weather data from WeatherAPI
def get_weather(location):
    weather_url = f"http://api.weatherapi.com/v1/current.json?key={WEATHER_API_KEY}&q={location}"
    response = requests.get(weather_url)

    if response.status_code == 200:
        data = response.json()
        weather_condition = data['current']['condition']['text']
        temperature = data['current']['temp_c']
        return weather_condition, temperature
    else:
        print("Error fetching weather data")
        return None, None

# Step 2: Get exercises from Wger API with English language
def get_exercises(workout_type):
    # Adding the language parameter for English (language=2)
    exercise_url = f"https://wger.de/api/v2/exercise/?category={workout_type}&language=2"
    response = requests.get(exercise_url)

    if response.status_code == 200:
        exercises = response.json()['results']
        return exercises
    else:
        print("Error fetching exercise data")
        return []

# Step 3: Suggest exercises based on weather with filtering for English exercises and removing HTML tags
def suggest_workout(weather_condition):
    if "Clear" in weather_condition or "Sunny" in weather_condition:
        print("Weather is clear, recommending outdoor exercises.")
        exercises = get_exercises(9)  # Outdoor exercises (Cardio, assuming category ID 9)
    else:
        print("Weather is not favorable, recommending indoor exercises.")
        exercises = get_exercises(8)  # Indoor exercises (Strength, assuming category ID 8)

    # Filter out non-English exercises by checking common German characters or phrases
    english_exercises = [ex for ex in exercises if not any(char in ex['description'] for char in ['ä', 'ö', 'ü', 'ß', 'und', 'mit'])]

    # Display first 5 filtered exercises with HTML tags removed
    if english_exercises:
        print("Here are some recommended exercises in English:")
        for exercise in english_exercises[:5]:
            print(f"Exercise Name: {exercise['name']}")
            description = remove_html_tags(exercise['description'])
            print(f"Description: {description}\n")
    else:
        print("No English exercises found.")

# Main Function to run the exercise suggestion system
def main():
    location = input("Enter your location (city or zip code): ")
    weather_condition, temperature = get_weather(location)

    if weather_condition:
        print(f"Current weather: {weather_condition}, Temperature: {temperature}°C")
        suggest_workout(weather_condition)
    else:
        print("Could not retrieve weather data")

# Run the main function
if __name__ == "__main__":
    main()


Enter your location (city or zip code): 10013
Current weather: Sunny, Temperature: 25.6°C
Weather is clear, recommending outdoor exercises.
Here are some recommended exercises in English:
Exercise Name: Alternate back lunges
Description: The posterior muscles of the buttocks, hamstrings, soleus and gastrocnemius are trained more

Exercise Name: Arabesque
Description: 
  

    

  
  
  Take all your weight onto one&nbsp;leg&nbsp;and you're going to&nbsp;maintain that position, keeping your hips and pelvis level the whole time. With&nbsp;your back in a neutral position you want to tilt yourself forward kicking your leg back up and then slowly with your glutes bring yourself back up to neutral.
  


  

    

  

Exercise Name: Barbell Hack Squats
Description: Perform leg squats with barbell behind your legs

Exercise Name: Barbell Lunges Standing
Description: Put barbell on the back of your shoulders. Stand upright, then take the first step forward. Step should bring you forward so that